In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

import lightgbm as lgb

import sys
sys.path.append('../lightgbm')

In [2]:
from sklearn.metrics import mean_squared_error
import optuna

def preprocess(data):
    #add relative floor
    data['rel_floor'] = data['floor']/data['max_floor']
    #add relative kitch_sq
    data['relative_kitch_sq'] = data['kitch_sq']/data['full_sq']
    #add room size
    data['room_size'] = data['life_sq']/data['num_room']
    # add month and day of week
    data['month'] = data.timestamp.dt.month.astype(int)
    data['day_of_week'] = data.timestamp.dt.dayofweek.astype(int)
    
    data['bought_minus_built'] = data.timestamp.dt.year.astype(int) - data['build_year']
    
    data.loc[data['full_sq']==data['life_sq'],'life_sq'] = np.nan

    return data
def fact_binary(data):
    data = data.applymap(lambda x: x if x!='yes' else 1)
    data = data.applymap(lambda x: x if x!='no' else 0)
    data = data.applymap(lambda x: x if x!='OwnerOccupier' else 0)
    data = data.applymap(lambda x: x if x!='Investment' else 1)
    return data

class sep_estimator:
    def __init__(self,owner_params,investment_params):
        self.est1 = lgb.LGBMRegressor(**owner_params)
        self.est2 = lgb.LGBMRegressor(**investment_params)
        
    def preprocess_owner(self,data,mode):
        assert (data['product_type'].values==0).all()
        if mode=='predict':
            data.loc[data['full_sq'].isnull(),'full_sq'] = 50
        data = data.drop('timestamp',axis=1)
        return data
    
    def preprocess_investment(self,data,mode):
        if mode=='predict':
            assert (data['product_type'].values==1).all()
            data.loc[data['full_sq'].isnull(),'full_sq'] = 50
        data = data.drop('timestamp',axis=1)
        return data
    
    def tuneHyperParams(self,X,y,size):
        def objective(trial):
            if (size == 'small'):
                params = {
                    "objective": "regression",
                    "metric": "rmse",
                    "verbosity": -1,
                    "n_estimators": trial.suggest_int("n_estimators", 100, 500),
    #                 "boosting": trial.suggest_categorical("boosting", ["gbdt", "rf", "dart"]),
                    "lambda_l2": trial.suggest_float("lambda_l2", 0, 10),
                    "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1),
                    "bagging_freq": 1,
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**8),
                    "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
                    "max_depth": trial.suggest_int("max_depth", 5, 10),
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "subsample": trial.suggest_float("subsample", 0.01, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 50),
                    "max_bin": trial.suggest_int("max_bin", 128, 512),
                }
            else:
                params = {
                    "objective": "regression",
                    "metric": "rmse",
                    "verbosity": -1,
                    "n_estimators": trial.suggest_int("n_estimators", 600, 1000),
    #                 "boosting": trial.suggest_categorical("boosting", ["gbdt", "rf", "dart"]),
                    "lambda_l2": trial.suggest_float("lambda_l2", 0, 10),
                    "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1),
                    "bagging_freq": 1,
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                    "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
                    "max_depth": trial.suggest_int("max_depth", 5, 15),
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "subsample": trial.suggest_float("subsample", 0.01, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
                    "max_bin": trial.suggest_int("max_bin", 128, 512),
                }

            model = lgb.LGBMRegressor(**params)
            model.fit(X_train, y_train, verbose=False)
            predictions = model.predict(X_val)
            rmse = mean_squared_error(y_val, predictions, squared=False)
            return rmse
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=50)
        
        print('Best hyperparameters:', study.best_params)
        print('Best RMSE:', study.best_value)
        return study.best_params
    
    def fit(self,X,y):
        X1 = X[X['product_type']==0]
        X2 = X
        X1 = self.preprocess_owner(X1,'train')
        y1 = y.loc[X1.index.values]/X1['full_sq']
        
        new_owner_params = self.tuneHyperParams(X1,y1, 'small')
        self.est1 = lgb.LGBMRegressor(**new_owner_params)
        
        X2 = self.preprocess_investment(X2,'train')
        y2 = y
        y2 = y2/X2['full_sq']
        
        new_investment_params = self.tuneHyperParams(X2,y2, 'large')
        self.est2 = lgb.LGBMRegressor(**new_investment_params)
        
        if len(X1)>0:
            self.est1.fit(X1,y1)
        if len(X2)>0:
            self.est2.fit(X2,y2)
    
    def predict(self,X):
        X1 = X[X['product_type']==0]
        X2 = X[X['product_type']==1]
        owner_index = X1.index.values
        investment_index = X.index.drop(owner_index).values
        X1 = self.preprocess_owner(X1,'predict')
        X2 = self.preprocess_investment(X2,'predict')
        res = pd.DataFrame(index=X.index)
        if len(X1)>0:
            pred1 = self.est1.predict(X1)
            res.loc[owner_index,0] = pred1*X1['full_sq']
        if len(X2)>0:
            pred2 = self.est2.predict(X2)
            res.loc[investment_index,0] = pred2*X2['full_sq']     
        return res[0].values.flatten()

In [3]:
data = pd.read_csv('./clean data/train_clean.csv',index_col='id',parse_dates=['timestamp'])
data = fact_binary(data)
data = preprocess(data)
data = data.drop(['sub_area','ecology'],axis=1)
data = data.loc[~data['full_sq'].isnull(),:]
train = data

C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='yes' else 1)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='no' else 0)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='OwnerOccupier' else 0)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='Investment' else 1)


In [4]:
#    https://www.kaggle.com/aharless/exercising-the-exorcism
#
#

rate_2015_q2 = 1
rate_2015_q1 = rate_2015_q2 / 0.9932
rate_2014_q4 = rate_2015_q1 / 1.0112
rate_2014_q3 = rate_2014_q4 / 1.0169
rate_2014_q2 = rate_2014_q3 / 1.0086
rate_2014_q1 = rate_2014_q2 / 1.0126
rate_2013_q4 = rate_2014_q1 / 0.9902
rate_2013_q3 = rate_2013_q4 / 1.0041
rate_2013_q2 = rate_2013_q3 / 1.0044
rate_2013_q1 = rate_2013_q2 / 1.0104  # This is 1.002 (relative to mult), close to 1:
rate_2012_q4 = rate_2013_q1 / 0.9832  #     maybe use 2013q1 as a base quarter and get rid of mult?
rate_2012_q3 = rate_2012_q4 / 1.0277
rate_2012_q2 = rate_2012_q3 / 1.0279
rate_2012_q1 = rate_2012_q2 / 1.0279
rate_2011_q4 = rate_2012_q1 / 1.076
rate_2011_q3 = rate_2011_q4 / 1.0236
rate_2011_q2 = rate_2011_q3 / 1
rate_2011_q1 = rate_2011_q2 / 1.011


# train 2015
train['average_q_price'] = 1

train_2015_q2_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2015_q2_index, 'average_q_price'] = rate_2015_q2

train_2015_q1_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2015_q1_index, 'average_q_price'] = rate_2015_q1


# train 2014
train_2014_q4_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2014_q4_index, 'average_q_price'] = rate_2014_q4

train_2014_q3_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2014_q3_index, 'average_q_price'] = rate_2014_q3

train_2014_q2_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2014_q2_index, 'average_q_price'] = rate_2014_q2

train_2014_q1_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2014_q1_index, 'average_q_price'] = rate_2014_q1


# train 2013
train_2013_q4_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2013_q4_index, 'average_q_price'] = rate_2013_q4

train_2013_q3_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2013_q3_index, 'average_q_price'] = rate_2013_q3

train_2013_q2_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2013_q2_index, 'average_q_price'] = rate_2013_q2

train_2013_q1_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2013_q1_index, 'average_q_price'] = rate_2013_q1


# train 2012
train_2012_q4_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2012_q4_index, 'average_q_price'] = rate_2012_q4

train_2012_q3_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2012_q3_index, 'average_q_price'] = rate_2012_q3

train_2012_q2_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2012_q2_index, 'average_q_price'] = rate_2012_q2

train_2012_q1_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2012_q1_index, 'average_q_price'] = rate_2012_q1


# train 2011
train_2011_q4_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2011_q4_index, 'average_q_price'] = rate_2011_q4

train_2011_q3_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2011_q3_index, 'average_q_price'] = rate_2011_q3

train_2011_q2_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2011_q2_index, 'average_q_price'] = rate_2011_q2

train_2011_q1_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2011_q1_index, 'average_q_price'] = rate_2011_q1

train['price_doc'] = train['price_doc'] * train['average_q_price']


#########################################################################################################


X = train.drop(["price_doc", "average_q_price"],axis=1)
y = train['price_doc']

C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\1443716078.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0068465565847766' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train.loc[train_2015_q1_index, 'average_q_price'] = rate_2015_q1


In [5]:
kaggle_test = pd.read_csv('./clean data/test_clean.csv', parse_dates=['timestamp'],index_col='id')
test1 = kaggle_test.copy()
test1 = preprocess(test1)
test1 = fact_binary(test1)
#test1 = test1.merge(macro,on='timestamp')
test1 = test1.drop(['ecology','sub_area'],axis=1)
test1.loc[test1['full_sq'].isnull(),'full_sq'] = 50
test1.loc[test1['product_type'].isnull(),'product_type'] = 1

C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='yes' else 1)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='no' else 0)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='OwnerOccupier' else 0)
C:\Users\Andrew\AppData\Local\Temp\ipykernel_3516\2463352502.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='Investment' else 1)


In [6]:
# 0.31010 params
owner_params = {
    'n_estimators': 388, 
    'lambda_l2': 5.8771302319480325, 
    'bagging_fraction': 0.8660359026008017, 
    'num_leaves': 202, 
    'feature_fraction': 0.7359742854201246, 
    'max_depth': 9, 
    'learning_rate': 0.08215092367951148, 
    'subsample': 0.41686599402329344, 
    'colsample_bytree': 0.1944406869188812, 
    'min_data_in_leaf': 11, 
    'max_bin': 392
}

investment_params = {
    'n_estimators': 640, 
    'lambda_l2': 6.421076660114255, 
    'bagging_fraction': 0.8195474653271653, 
    'num_leaves': 474, 
    'feature_fraction': 0.5006530112297926, 
    'max_depth': 10, 
    'learning_rate': 0.010757243865305521, 
    'subsample': 0.24435014285782658, 
    'colsample_bytree': 0.2998744009955994, 
    'min_data_in_leaf': 36, 
    'max_bin': 274
}

est = sep_estimator(owner_params=owner_params,investment_params=investment_params)
est.fit(X,y)

[I 2023-11-14 07:31:11,087] A new study created in memory with name: no-name-9cc4c708-611d-4275-81a5-3f7075677272
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.7671819568910898, colsample_bytree=0.4422032921125471 will be ignored. Current value: feature_fraction=0.7671819568910898
[LightGBM] [Warning] min_data_in_leaf is set=27, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=27
[LightGBM] [Warning] bagging_fraction is set=0.8983772472892508, subsample=0.7422927956260987 will be ignored. Current value: bagging_fraction=0.8983772472892508
[LightGBM] [Warning] lambda_l2 is set=1.5451618013220791, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5451618013220791
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-14 07:31:12,741] Trial 0 finished with value: 26663.787991699537 and parameters: {'n_estimators': 303, 'lambda_l2': 1.5451618013220791, 'bagging_fraction': 0.8983772472892508, 'num_leaves': 205, 'feature_fraction': 0.7671819568910898, 'max_depth': 7, 'learning_rate': 0.0027599198418127506, 'subsample': 0.7422927956260987, 'colsample_bytree': 0.4422032921125471, 'min_data_in_leaf': 27, 'max_bin': 173}. Best is trial 0 with value: 26663.787991699537.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.6269384779588572, colsample_bytree=0.515820060771841 will be ignored. Current value: feature_fraction=0.6269384779588572
[LightGBM] [Warning] min_data_in_leaf is set=31, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=31
[LightGBM] [Warning] bagging_fraction is set=0.6734266637171888, subsample=0.7136527901381782 will be ignored. Current value: bagging_fraction=0.6734266637171888
[LightGBM] [Warning] lambda_l2 is set=4.111772170819834, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.111772170819834
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-14 07:31:13,722] Trial 1 finished with value: 21100.629482350112 and parameters: {'n_estimators': 480, 'lambda_l2': 4.111772170819834, 'bagging_fraction': 0.6734266637171888, 'num_leaves': 49, 'feature_fraction': 0.6269384779588572, 'max_depth': 6, 'learning_rate': 0.02267871066816075, 'subsample': 0.7136527901381782, 'colsample_bytree': 0.515820060771841, 'min_data_in_leaf': 31, 'max_bin': 359}. Best is trial 1 with value: 21100.629482350112.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Warning] feature_fraction is set=0.5903990430353143, colsample_bytree=0.8255920483702012 will be ignored. Current value: feature_fraction=0.5903990430353143
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] bagging_fraction is set=0.9720717537394429, subsample=0.8793564569423541 will be ignored. Current value: bagging_fraction=0.9720717537394429
[LightGBM] [Warning] lambda_l2 is set=9.47781656266903, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.47781656266903
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-14 07:42:11,892] Trial 48 finished with value: 38485.466166527 and parameters: {'n_estimators': 876, 'lambda_l2': 9.47781656266903, 'bagging_fraction': 0.9720717537394429, 'num_leaves': 367, 'feature_fraction': 0.5903990430353143, 'max_depth': 10, 'learning_rate': 0.005216849178715095, 'subsample': 0.8793564569423541, 'colsample_bytree': 0.8255920483702012, 'min_data_in_leaf': 41, 'max_bin': 146}. Best is trial 33 with value: 38358.47512708974.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.6774395466933044, colsample_bytree=0.9589587581031029 will be ignored. Current value: feature_fraction=0.6774395466933044
[LightGBM] [Warning] min_data_in_leaf is set=66, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=66
[LightGBM] [Warning] bagging_fraction is set=0.7817085130448806, subsample=0.8151795563646692 will be ignored. Current value: bagging_fraction=0.7817085130448806
[LightGBM] [Warning] lambda_l2 is set=9.996428539218133, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.996428539218133
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-14 07:42:24,117] Trial 49 finished with value: 38473.87042981599 and parameters: {'n_estimators': 922, 'lambda_l2': 9.996428539218133, 'bagging_fraction': 0.7817085130448806, 'num_leaves': 618, 'feature_fraction': 0.6774395466933044, 'max_depth': 13, 'learning_rate': 0.010528239772901673, 'subsample': 0.8151795563646692, 'colsample_bytree': 0.9589587581031029, 'min_data_in_leaf': 66, 'max_bin': 251}. Best is trial 33 with value: 38358.47512708974.


Best hyperparameters: {'n_estimators': 892, 'lambda_l2': 9.120174322435526, 'bagging_fraction': 0.8212524434096794, 'num_leaves': 312, 'feature_fraction': 0.6864365936394377, 'max_depth': 10, 'learning_rate': 0.009025702249254646, 'subsample': 0.7227675964333954, 'colsample_bytree': 0.8500717211078491, 'min_data_in_leaf': 47, 'max_bin': 186}
Best RMSE: 38358.47512708974
[LightGBM] [Warning] feature_fraction is set=0.8208352381370674, colsample_bytree=0.9891475516872527 will be ignored. Current value: feature_fraction=0.8208352381370674
[LightGBM] [Warning] min_data_in_leaf is set=16, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=16
[LightGBM] [Warning] bagging_fraction is set=0.9108190713445593, subsample=0.7251169328279444 will be ignored. Current value: bagging_fraction=0.9108190713445593
[LightGBM] [Warning] lambda_l2 is set=0.36978134386024575, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.36978134386024575


In [8]:
pred_test = np.zeros(shape=test1.values.shape[0])

pred_test = est.predict(test1)

pred_test = pred_test * .969 + 10
# pred_test *= 0.9915
subm = pd.read_csv('../../Dataset/sample_submission.csv/sample_submission.csv')
subm['price_doc'] = pred_test
subm.to_csv('./clean data/output.csv',index=False)
subm.head()

,id,price_doc
0,30474,5.455544e+06
1,30475,8.213875e+06
2,30476,5.528125e+06
3,30477,6.540667e+06
4,30478,5.083761e+06
